# Load Overture Data
## Query names and categories w/o bounding box from overture

In [61]:
# %%skip
# %pip install ipython-sql
# %pip install geopandas
# %pip install duckdb
# %load_ext sql
# %pip install duckdb-engine

In [ ]:
from IPython.core.magic import register_cell_magic
import pandas as pd
import geopandas as gpd
import duckdb
from shapely import wkt
import csv


Get overture categories in a list

In [63]:
with open('data/overture_categories.csv', 'r') as f:
    # Try with a semicolon delimiter; change to '\t' for tab or another if needed
    categories = pd.read_csv(f, delimiter=';')

print("Loading data from DuckDB...")
print(categories.to_string())

Loading data from DuckDB...
                                                Category code                                                                                                                                                                    Overture Taxonomy
0                                               eat_and_drink                                                                                                                                                                      [eat_and_drink]
1                                                  restaurant                                                                                                                                                           [eat_and_drink,restaurant]
2                                           afghan_restaurant                                                                                                                                         [eat_and_drink,restaurant,afghan_restaurant]


In [64]:
%reload_ext sql
%sql duckdb://
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

%sql INSTALL spatial;
%sql INSTALL httpfs;
%sql LOAD spatial;
%sql LOAD httpfs;
%sql SET s3_region='us-west-2'


""


In [65]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

- This part should output each category from overture categories.csv file
- SQL script to parse locations based on categories from a particular metropolitan region

In [ ]:
%%sql gulf_water <<
SELECT 
    id, 
    names.primary AS primary_name,
    ST_AsText(geometry) as geometry
FROM 
    read_parquet('s3://overturemaps-us-west-2/release/2024-09-18.0/theme=base/type=water/*', filename=true, hive_partitioning=1)
WHERE 
    bbox.xmin >= -91.3994
        and bbox.xmax <= -89.3864
        and bbox.ymin >= 29.152
        and bbox.ymax <= 30.5161


(duckdb.duckdb.ParserException) Parser Error: syntax error at or near "("
[SQL: SELECT
    id,
    names.primary AS primary_name,
    categories
    ST_AsText(geometry) as geometry
FROM
    read_parquet('s3://overturemaps-us-west-2/release/2024-09-18.0/theme=base/type=water/*', filename=true, hive_partitioning=1)
WHERE
    bbox.xmin >= 41.55 
        and bbox.ymin <= -88.10]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [ ]:
gulf_water_gdf = gpd.GeoDataFrame(
    gulf_water,
    geometry=gulf_water['geometry'].apply(wkt.loads),
    crs="EPSG:4326"
)

NameError: name 'chicago_metro' is not defined

In [ ]:
%%sql get_businesses_in_area <<
SELECT
    id,
    names.primary AS primary_name,
    categories
    ST_AsText(geometry) as geometry
FROM
    read_parquet('s3://overturemaps-us-west-2/release/2024-09-18.0/theme=base/type=water/*', filename=true, hive_partitioning=1)
WHERE
    bbox.xmin >= 41.55 
        and bbox.ymin <= -88.10


In [ ]:
%%sql gulf_water <<
SELECT 
    id, 
    names.primary AS primary_name,
    ST_AsText(geometry) as geometry
FROM 
    read_parquet('s3://overturemaps-us-west-2/release/2024-09-18.0/theme=base/type=water/*', filename=true, hive_partitioning=1)
WHERE 
    bbox.xmin >= -91.3994
        and bbox.xmax <= -89.3864
        and bbox.ymin >= 29.152
        and bbox.ymax <= 30.5161

In [70]:
# 1) turn the WKT‐strings into shapely geometries
chicago_metro['geometry'] = chicago_metro['geometry'].apply(wkt.loads)

# 2) wrap the DataFrame as a GeoDataFrame
chicago_metro_gdf = gpd.GeoDataFrame(
    chicago_metro,
    geometry='geometry',
    crs='EPSG:4326'
)

NameError: name 'chicago_metro' is not defined